In [1]:
!jupyter kernelspec list

Available kernels:
  python3    /vol/bitbucket/ss5120/nlp_env/share/jupyter/kernels/python3


In [2]:
USERNAME = 'ss5120'
content_path = f'/vol/bitbucket/{USERNAME}/NLP_CW/'
data_path = f'/vol/bitbucket/{USERNAME}/NLP_CW/data/'

In [3]:
!nvidia-smi

Fri Feb 23 10:54:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.146.02             Driver Version: 535.146.02   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


|   0  NVIDIA GeForce RTX 4080        Off | 00000000:2D:00.0  On |                  N/A |
| 31%   38C    P2              38W / 320W |  14613MiB / 16376MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|    0   N/A  N/A   2462615      G   /usr/lib/xorg/Xorg                           88MiB |
|    0   N

In [4]:
!pip install simpletransformers
!pip install tensorboardx
!pip install sklearn
!pip install torch

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [5]:
from simpletransformers.classification import (
    ClassificationModel,
    ClassificationArgs,
    MultiLabelClassificationModel,
    MultiLabelClassificationArgs,
)
from urllib import request
import pandas as pd
import logging
import torch
import numpy as np
from collections import Counter
from ast import literal_eval
from sklearn.metrics import (
    f1_score,
    confusion_matrix,
    precision_score,
    recall_score,
    accuracy_score,
)
from sklearn.utils import shuffle

/vol/bitbucket/ss5120/nlp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print("There are %d GPU(s) available." % torch.cuda.device_count())

    print("We will use the GPU:", torch.cuda.get_device_name(0))

# If not...
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA GeForce RTX 4080


In [15]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split("/")[-1]
print(f"Fetching {module_url}")
# with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name, "w") as outf:
    a = f.read()
    outf.write(a.decode("utf-8"))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [19]:
from dont_patronize_me import DontPatronizeMe

In [20]:
dpm = DontPatronizeMe('.', data_path + 'task4_test.tsv')

In [21]:
dpm.load_task1()

In [22]:
train_labels = pd.read_csv(data_path + "train_semeval_parids-labels.csv")
dev_labels = pd.read_csv(data_path + "dev_semeval_parids-labels.csv")

In [23]:
train_labels.head()

,par_id,label
0,4341,"[1, 0, 0, 1, 0, 0, 0]"
1,4136,"[0, 1, 0, 0, 0, 0, 0]"
2,10352,"[1, 0, 0, 0, 0, 1, 0]"
3,8279,"[0, 0, 0, 1, 0, 0, 0]"
4,1164,"[1, 0, 0, 1, 1, 1, 0]"


In [24]:
train_labels.par_id = train_labels.par_id.astype(str)
dev_labels.par_id = dev_labels.par_id.astype(str)

In [26]:
train_labels.par_id

0        4341
1        4136
2       10352
3        8279
4        1164
        ...  
8370     8380
8371     8381
8372     8382
8373     8383
8374     8384
Name: par_id, Length: 8375, dtype: object

In [27]:
dev_labels.par_id

0        4046
1        1279
2        8330
3        4063
4        4089
        ...  
2089    10462
2090    10463
2091    10464
2092    10465
2093    10466
Name: par_id, Length: 2094, dtype: object

In [28]:
train_set = []

for i in range(len(train_labels)):
    par_id = train_labels.par_id[i]
    sentence = dpm.train_task1_df.loc[
        dpm.train_task1_df.par_id == par_id
    ].text.values[0]
    label = dpm.train_task1_df.loc[
        dpm.train_task1_df.par_id == par_id
    ].label.values[0]
    train_set.append({"par_id": par_id, "sentence": sentence, "label": label})

In [29]:
train_set = pd.DataFrame(train_set)

In [34]:
print('No. of PCL: ' + str(len(train_set[train_set.label==1])))
print('No. of non-PCL: ' + str(len(train_set[train_set.label==0])))

No. of PCL: 794
No. of non-PCL: 7581


In [35]:
test_set = []

for i in range(len(dev_labels)):
    par_id = dev_labels.par_id[i]
    sentence = dpm.train_task1_df.loc[
        dpm.train_task1_df.par_id == par_id
    ].text.values[0]
    label = dpm.train_task1_df.loc[
        dpm.train_task1_df.par_id == par_id
    ].label.values[0]
    test_set.append({"par_id": par_id, "sentence": sentence, "label": label})

In [36]:
test_set = pd.DataFrame(test_set)

In [37]:
print('No. of PCL: ' + str(len(test_set[test_set.label==1])))
print('No. of non-PCL: ' + str(len(test_set[test_set.label==0])))

No. of PCL: 199
No. of non-PCL: 1895
